# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.stem import WordNetLemmatizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
le = WordNetLemmatizer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [le.lemmatize(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

body_len  punct%    0    1    2    3    4    5    6    7  ...  8901  8902  \
0       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1        49     4.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2        62     3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3        28     7.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4       135     4.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

   8903  8904  8905  8906  8907  8908  8909  8910  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8913 columns]

### Build our own Grid-search

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_features.values, data['label'], test_size=0.2)

In [13]:
 def train_RF(n_est, depth):
        rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
        rf_model = rf.fit(X_train, y_train)
        y_pred = rf_model.predict(X_test)
        precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
        print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'
              .format(n_est, depth, round(precision, 3), round(recall, 3),
              round(((y_pred==y_test).sum() / len(y_pred)), 3) * 100))

In [14]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.265 / Accuracy: 90.0
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.543 / Accuracy: 93.8
Est: 10 / Depth: 30 ---- Precision: 0.991 / Recall: 0.728 / Accuracy: 96.2
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.808 / Accuracy: 97.39999999999999
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.219 / Accuracy: 89.4
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.543 / Accuracy: 93.8
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.709 / Accuracy: 96.1
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.834 / Accuracy: 97.8
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.192 / Accuracy: 89.0
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.576 / Accuracy: 94.3
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.709 / Accuracy: 96.1
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.848 / Accuracy: 97.89999999999999
